End-to-end data processing from data downloading to 1-second data segmentation generation

# Install and Import Libraries

In [1]:
!pip install vitaldb neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.4/708.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 wh

In [32]:
import requests
import pandas as pd
import vitaldb
import io
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
import neurokit2 as nk

from scipy import signal
from scipy.spatial.distance import cosine
from scipy.signal import find_peaks

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# User-Defined Parameters
* All urls used in data downloading
* All filenames and paths
* All parameters used in data processing

In [22]:
RAWDATAPATH = 'drive/MyDrive/2025_PPG_GLUC/Data/Raw Data/'
VITALDB_DATAPATH = 'drive/MyDrive/2025_PPG_GLUC/Data/Raw Data/VitalDB/'
OUTPUT_PATH = 'drive/MyDrive/2025_PPG_GLUC/Data/Processed Data/BW_ppg_16min_windows/'

INPUT_PATH = 'drive/MyDrive/2025_PPG_GLUC/Data/Processed Data/BW_ppg_16min_windows/'
OUTPUT_PATH2 = 'drive/MyDrive/2025_PPG_GLUC/Data/Processed Data/BW_ppg_16min_filtered/'
OUTPUT_PATH3 = 'drive/MyDrive/2025_PPG_GLUC/Data/Final Data/1s_segmentation/' # Final Output

# Signal parameters
FS = 100  # Sampling frequency (Hz)
WINDOW_SIZE = 100  # 1 second = 100 samples

# Thresholds from paper
HEIGHT_THRESHOLD = 20
DISTANCE_THRESHOLD = int(0.8 * FS)  # 0.8s min between peaks
SIMILARITY_THRESHOLD = 0.85

In [6]:
os.listdir(RAWDATAPATH)

['clinical_parameters.csv',
 'DataSource.txt',
 'lab_parameters.csv',
 'track_names.csv',
 'VitalDB',
 'lab_data.csv',
 'clinical_data.csv']

# Download Data (One-Time)

There are discrepancies in clinical data (at least in age) being found between url version and flatfile version

Use url version which is close to what paper uses

## Lab and Clinical Data

In [ ]:
# # Download Lab Data
# labs_url = "https://api.vitaldb.net/labs"
# labs_response = requests.get(labs_url)

# if labs_response.status_code != 200:
#     print(f"Failed to load labs data: {labs_response.status_code}")
# else:
#     labs_data = pd.read_csv(io.StringIO(labs_response.text))
#     print(f"Total labs measurements: {len(labs_data)}")
#     labs_data.to_csv(RAWDATAPATH + 'lab_data.csv')

# # Download Clinical Data
# clinical_url = "https://api.vitaldb.net/cases"
# clinical_response = requests.get(clinical_url)

# if clinical_response.status_code != 200:
#     print(f"Failed to load clinical data: {clinical_response.status_code}")
# else:
#     clinical_data = pd.read_csv(io.StringIO(clinical_response.text))
#     print(f"Clinical data loaded: {len(clinical_data)} cases")
#     clinical_data.to_csv(RAWDATAPATH + 'clinical_data.csv')

## VitalDB

In [ ]:
# # Find Cases with Glucose Lab Data
# lab = pd.read_csv(RAWDATAPATH + 'lab_data.csv')
# lab_gluc = lab[lab['name'] == 'gluc']
# lab_gluc_cases = lab_gluc['caseid'].unique().tolist()

# # Obtain PLETH (PPG) Data: Original PPG is 500 Hz, need to downsample to 100 Hz (from paper)
# tracklist = ['SNUADC/PLETH']
# existlist = [int(i.split('_')[-1].split('.')[0]) for i in os.listdir(VITALDB_DATAPATH + 'Vital/')]
# to_retrieve = list(set(lab_gluc_cases) - set(existlist))
# for caseid in tqdm(to_retrieve):
#     tmp = vitaldb.load_case(caseid, tracklist, interval=1/100)
#     np.save(VITALDB_DATAPATH + 'Vital/' + f'vitaldb_PLETH_case_{caseid}.npy', tmp)

# # Re-try for those size ~1 KB
# tracklist = ['SNUADC/PLETH']
# existlist = [int(i.split('_')[-1].split('.')[0]) for i in os.listdir(VITALDB_DATAPATH + 'Vital/')]
# original_list = [caseid for caseid in existlist if os.path.getsize(
#     VITALDB_DATAPATH + 'Vital/' + f'vitaldb_PLETH_case_{caseid}.npy')/1024 <= 1]
# print(len(original_list))

# for caseid in tqdm(original_list):
#     tmp = vitaldb.load_case(caseid, tracklist, interval=1/100)
#     np.save(VITALDB_DATAPATH + 'Vital/' + f'vitaldb_PLETH_case_{caseid}.npy', tmp)

# # Check again
# after_list = [caseid for caseid in existlist if os.path.getsize(
#     VITALDB_DATAPATH + 'Vital/' + f'vitaldb_PLETH_case_{caseid}.npy')/1024 <= 1]
# print(len(after_list))


# PPG Temporal Alignment
* Find Caseids with Glucose Reading
* Filter Glucose Reading by Case Timings
* Generate PPG alignment

In [15]:
labs_data = pd.read_csv(RAWDATAPATH + 'lab_data.csv').drop(columns=['Unnamed: 0'])
clinical_data = pd.read_csv(RAWDATAPATH + 'clinical_data.csv').drop(columns=['Unnamed: 0'])

labs_data.shape, clinical_data.shape, clinical_data['caseid'].nunique()

((928448, 4), (6388, 74), 6388)

## Find Caseids with Glucose Reading

In [16]:
glucose_data = labs_data[labs_data['name'] == 'gluc'].copy()
print(glucose_data.shape, glucose_data['caseid'].nunique())

# Glucose reading outside case time
print(glucose_data[glucose_data['dt']<0].shape[0]/glucose_data.shape[0])

(35358, 4) 5091
0.255218055319871


## Filter Glucose Reading by Case Timings
Filters applied:
* dt >= casestart
* dt <= caseend

In [17]:
glucose_with_timing = pd.merge(glucose_data,
                               clinical_data[['caseid', 'casestart', 'caseend']],
                               on='caseid', how='left')
print(clinical_data.shape, glucose_with_timing.shape)
print(clinical_data[clinical_data['casestart'].isna()].shape,
      clinical_data[clinical_data['caseend'].isna()].shape,
      glucose_with_timing[glucose_with_timing['dt'].isna()].shape,
      glucose_with_timing['caseid'].nunique())

# Valid time
valid_timing = glucose_with_timing[
        (glucose_with_timing['dt'] >= glucose_with_timing['casestart']) &
        (glucose_with_timing['dt'] <= glucose_with_timing['caseend'])
    ]
print(valid_timing.shape, valid_timing['caseid'].nunique())

(6388, 74) (35358, 6)
(0, 74) (0, 74) (0, 6) 5091
(8847, 6) 3297


## Generate PPG Temporal Alignment
*   Load PPG data from .npy files by case ID
*   Extract 16-minute windows
*   Temporal Alignment using timestamps

**Temporal Alignment Calculated Using**:
* `time_offset = glucose_time - case_start` - Time of glucose measurement relative to case start (in seconds)
* `center_sample = int(time_offset * sampling_rate)` - Exact sample index where glucose was measured
* `half_window = window_samples // 2` - Half of 16-minute window = 48,000 samples (8 minutes)
* `start_idx = center_sample - half_window` - Start of window (8 minutes before glucose measurement)
* `end_idx = center_sample + half_window` - End of window (8 minutes after glucose measurement)
* `ppg_window = ppg_data[start_idx:end_idx]` - Extract 96,000 samples (16 minutes at 100 Hz)

**Which cases are removed / avoided?**:
* Any case that the glucose reading is not within the case start / case end is not used
* Any case that has the glucose reading too close to the case start / case end where it is not able to extract the full 16 minute window
* Any case that has missing PPG data
* Any case that has misisng case timing information

In [ ]:
# Sort caseid
valid_timing = valid_timing.sort_values(by='caseid')

# Parameters
sampling_rate = 100
window_duration = 16 * 60  # seconds
window_samples = window_duration * sampling_rate
caseid_holder = int(-1)

# Generate meta data for tracking purpose
ppg_data_meta = pd.DataFrame({'Caseid':[], 'Gluc':[], 'Info':[]})

for index, row in tqdm(valid_timing.iterrows()):
    caseid = row['caseid']
    glucose_time = row['dt']
    case_start = row['casestart']
    filename = f"case_{caseid}_time_{glucose_time}_cleaned.npy"

    # Skip if already exist
    if os.path.exists(OUTPUT_PATH + filename):
        continue

    # Load Data
    if caseid != caseid_holder:  # Reload
        ppg_file = f"{RAWDATAPATH}VitalDB/Vital/vitaldb_PLETH_case_{caseid}.npy"
        try:
            ppg_data = np.load(ppg_file)
            caseid_holder = caseid
        except:
            print(f'Error in loading vitaldb_PLETH_case_{caseid}.npy')
            continue  # skip to next

    # Extract window
    time_offset = glucose_time - case_start # to calculate relative time to case start
    center_sample = int(time_offset * sampling_rate) # to find the sampling index for gluc reading
    half_window = window_samples // 2 # for the 8 minute window

    start_idx = center_sample - half_window # start of window
    end_idx = center_sample + half_window # end of window

    if start_idx < 0 or end_idx >= len(ppg_data): # if ppg data out of the boundary, returns none
        print(f'Error in extracting {caseid}: out of boundary')
        continue
    else: # Forward filling missing
        cleaned_ppg_data = pd.Series(ppg_data[start_idx:end_idx].flatten()).ffill().values
        np.save(OUTPUT_PATH + filename, cleaned_ppg_data)

    # Note down the missing values that being forward filled
    original_nan = sum([1 for i in ppg_data.flatten() if np.isnan(i)])
    filled_nan = sum([1 for i in cleaned_ppg_data.flatten() if np.isnan(i)])
    ppg_data_meta = pd.concat([ppg_data_meta,
        pd.DataFrame({'Caseid':[caseid], 'Gluc': [glucose_time],
                      'Info': [f'Original missing: {original_nan}, After filling missing: {filled_nan}']})],
        ignore_index=True)

ppg_data_meta.to_csv(OUTPUT_PATH + 'BW_ppg_16min_metadata.csv', index=False)
ppg_data_meta.shape

# Filter PPG
* Butterworth filter

**Nyquist Sampling Theorem**: https://www.geeksforgeeks.org/electronics-engineering/nyquist-sampling-theorem/

To accurately capture a signal, your sampling frequency must be at least twice the highest frequency component in that signal.

Otherwise, it may have band aliasing, i.e. high-freq signal appears like low-freq signal, causing distortion

In [ ]:
# Parameters
sampling_rate = 100
low_cut = 0.5
high_cut = 8
order = 3

# Check the remaining
tmp_ppg_data_meta = ppg_data_meta[(~ppg_data_meta['Info'].str.contains('Out of boundary')) & (
    ~ppg_data_meta['Info'].str.contains('After filling missing: 96000'))].copy()
tmp_ppg_data_meta['filename'] = tmp_ppg_data_meta.apply(
    lambda x: f"case_{int(x['Caseid'])}_time_{int(x['Gluc'])}_filtered.npy", axis=1)

for index, row in tqdm(tmp_ppg_data_meta[~tmp_ppg_data_meta['filename'].isin(
    os.listdir(OUTPUT_PATH2))].iterrows()):
    if (row['Info'] == 'Out of boundary') | ('After filling missing: 96000' in row['Info']):
        continue
    else:
        caseid = int(row['Caseid'])
        glucose_time = int(row['Gluc'])
        ppg_file = f"{INPUT_PATH}case_{caseid}_time_{glucose_time}_cleaned.npy"
        output_file = f"{OUTPUT_PATH2}case_{caseid}_time_{glucose_time}_filtered.npy"
        try:
            ppg_data = np.load(ppg_file)
            ppg_filtered = nk.signal_filter(
                ppg_data,
                sampling_rate=sampling_rate,
                lowcut=low_cut,       # low cut frequency (Hz)
                highcut=high_cut,        # high cut frequency (Hz)
                method='butterworth',
                order=order
            )
            np.save(output_file, ppg_filtered)
        except:
            print(f'Error in loading {ppg_file}')


# Generate 1-Second Segmentation
* Peak detection on 16 min window to find all systolic peaks
* Extract 1s windows on each peak
* Create the template (mean of all windows per case)
* Perform Cosine similarity filtering (threshold at 0.85)
* Height threshold at 20 amplitude
* Distance threshold set to 0.8s min between peaks

In [23]:
# Get all filtered files
ppg_files = sorted(Path(OUTPUT_PATH2).glob('case_*_filtered.npy'))
print(f"Found {len(ppg_files)} files to process\n")

Found 7300 files to process



## Prepare Mini-Functions

Using scipy's find_peaks, prominence set to 0

### Peak Detection

In [25]:
def detect_peaks(ppg_signal, height_threshold=HEIGHT_THRESHOLD,
                 distance_threshold=DISTANCE_THRESHOLD, sampling_rate=FS):
    peaks, _ = find_peaks(
        ppg_signal,
        height=height_threshold,
        distance=distance_threshold
    )

    return peaks

### Extract Windows
- Extracts 1s windows centered on each peak
- Only keep windows that contain exactly 1 peak
- window_size set to 100 samples (1s)
- Returns: list of valid windows (windows)

In [26]:
def extract_windows(ppg_signal, peaks, window_size):
    windows = []
    half_window = window_size // 2  # 50 samples

    for peak in peaks:
        # Calculate window boundaries
        window_start = max(0, peak - half_window)
        window_end = min(len(ppg_signal), peak + half_window)

        # Extract window
        window = ppg_signal[window_start:window_end]

        # Skip if window is not full size or contains NaN
        if len(window) != window_size or np.any(np.isnan(window)):
            continue

        # Count peaks in this window
        peak_count = count_peaks_in_window(window, height_threshold=HEIGHT_THRESHOLD)

        # Only keep if exactly 1 peak
        if peak_count == 1:
            windows.append(window)

    return windows

### Count Peaks in Window

In [27]:
def count_peaks_in_window(window, height_threshold):

    count = 0

    for i in range(1, len(window) - 1):
        if window[i-1] < window[i] > window[i+1]:
            if window[i] > height_threshold:
                count += 1

    return count

### Computing Template

According to the pseudocode by the paper, they calculate template using mean of 16 min window

In [28]:
def compute_template(windows):

    if len(windows) == 0:
        return None

    template = np.mean(windows, axis=0)
    return template

### Cosine Similarity
Computes Cosine Similarity between window and template

In [29]:
def cosine_similarity(window, template):

    if template is None or len(window) != len(template):
        return 0.0

    # Using scipy's cosine distance: similarity = 1 - distance
    similarity = 1 - cosine(window, template)

    return similarity

## Filter windows by similarity
Filters windows by cosine similairity to template

In [30]:
def filter_windows_by_similarity(windows, template, similarity_threshold):

    filtered_windows = []

    for window in windows:
        similarity = cosine_similarity(window, template)

        if similarity >= similarity_threshold:
            filtered_windows.append(window)

    return filtered_windows


## Run All 1-Second Segmentation Process

In [33]:
def main_process_window(ppg_signal):

    # Step 1: Detect peaks
    peaks = detect_peaks(ppg_signal, HEIGHT_THRESHOLD, DISTANCE_THRESHOLD)

    # Step 2: Extract windows centered on peaks
    windows = extract_windows(ppg_signal, peaks, WINDOW_SIZE)

    # Step 3: Compute template
    template = compute_template(windows)

    # Step 4: Filter by similarity
    if template is not None:
        filtered_windows = filter_windows_by_similarity(windows, template, SIMILARITY_THRESHOLD)
    else:
        filtered_windows = []

    # Statistics
    stats = {
        'n_peaks': len(peaks),
        'n_windows_extracted': len(windows),
        'n_windows_filtered': len(filtered_windows),
        'rejection_rate': 1 - (len(filtered_windows) / len(peaks)) if len(peaks) > 0 else 1.0
    }

    return filtered_windows, stats

In [ ]:
Path(OUTPUT_PATH3).mkdir(parents=True, exist_ok=True)

ppg_files = sorted(Path(OUTPUT_PATH2).glob('case_*_filtered.npy'))
print(f"Found {len(ppg_files)} files to process\n")

results = []

for ppg_file in tqdm(ppg_files, desc="Processing files"):
    # Parse filename
    parts = ppg_file.stem.split('_')
    caseid = int(parts[1])
    glucose_time = int(parts[3])

    # Load 16-minute PPG window
    try:
        ppg_16min = np.load(ppg_file)
    except (EOFError, ValueError, FileNotFoundError):
        results.append({
            'caseid': caseid,
            'glucose_time': glucose_time,
            'n_peaks': 0,
            'n_windows_extracted': 0,
            'n_windows_filtered': 0,
            'rejection_rate': 1.0
        })
        continue

    # Process this 16-min window
    filtered_windows, stats = main_process_window(ppg_16min)

    # Save valid segments if any
    if len(filtered_windows) > 0:
        segments_array = np.array(filtered_windows)  # Shape: (n_segments, 100)
        output_file = Path(OUTPUT_PATH3) / f'case_{caseid}_time_{glucose_time}_1s_segments.npy'
        np.save(output_file, segments_array)

    # Track results
    results.append({
        'caseid': caseid,
        'glucose_time': glucose_time,
        'n_peaks': stats['n_peaks'],
        'n_windows_extracted': stats['n_windows_extracted'],
        'n_windows_filtered': stats['n_windows_filtered'],
        'rejection_rate': stats['rejection_rate']
    })

# Meta data
summary_df = pd.DataFrame(results)
summary_df.to_csv(Path(OUTPUT_PATH3).parent / '1s_seg_metadata.csv', index=False)

In [46]:
np.array(filtered_windows).shape

(791, 100)